In [1]:
import pandas as pd
import multiprocessing
import numpy as np
import os
import random
from numpy import array
from numpy import mean 
#import guidedlda

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
age_df = pd.read_csv('/content/drive/MyDrive/CS640/final_project/preprocessed_tweets_with_for_age_pred.csv',  lineterminator='\n')
## drop NaN
age_df.dropna(inplace=True)

In [4]:
## sample from age_df
freq = pd.DataFrame({'label':[0, 1],
                     'nostoextract':[36115, 36115], })
def bootstrap(data, freq):
    freq = freq.set_index('label')

    # This function will be applied on each group of instances of the same
    # class in `data`.
    def sampleClass(classgroup):
        cls = classgroup['label'].iloc[0]
        nDesired = freq.nostoextract[cls]
        nRows = len(classgroup)

        nSamples = min(nRows, nDesired)
        return classgroup.sample(nSamples)

    samples = data.groupby('label').apply(sampleClass)

    # If you want a new index with ascending values
    # samples.index = range(len(samples))

    # If you want an index which is equal to the row in `data` where the sample
    # came from
    samples.index = samples.index.get_level_values(1)

    # If you don't change it then you'll have a multiindex with level 0
    # being the class and level 1 being the row in `data` where
    # the sample came from.

    return samples

sampled_age_df = bootstrap(age_df,freq)

In [5]:
## stem 
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [6]:
stem_list = []
for txt in sampled_age_df['text']:
    wrds = txt.split()
    stem_wrds = []
    
    for i in wrds:
        stem_wrds.append(stemmer.stem(i))
    
    str1 = ' '.join(stem_wrds)  
    stem_list.append(str1)

sampled_age_df['text'] = stem_list

In [7]:
sampled_age_df['label'].value_counts()

1    36115
0    36115
Name: label, dtype: int64

In [ ]:
!pip install allennlp==0.9.0.
!pip install flair==0.8

In [9]:
from torch.optim.adam import Adam
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from pathlib import Path
from flair.data import Sentence
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.embeddings import TransformerDocumentEmbeddings
from flair.embeddings import FlairEmbeddings, BertEmbeddings, WordEmbeddings, DocumentRNNEmbeddings, ELMoEmbeddings

In [10]:
sampled_age_df

,Unnamed: 0,text,label,screen_name
78622,91672,pleas don touch raf,0,alli33s
82990,97057,weak for sure thank gabe miss you,0,MaiaBica
17570,19594,wa said didnt realiz,0,ay_p329
66226,76484,fact,0,mekaylaap
8781,9753,you can play the guitar solo hotel california ...,0,caro_number7
...,...,...,...,...
51952,59441,just post photo,1,Queen_Miasma
43595,49548,dad ask still keep contact with paramor how da...,1,koolkatkrystal
4717,5227,thi deadass wa dure the georgia trip,1,eccentric_mia_
15307,17007,thi sexi bitch keep bloom,1,nicholaslee137


In [11]:
sampled_age_df = sampled_age_df.drop(columns=['Unnamed: 0', 'screen_name'])
sampled_age_df['label'] = '__label__' + sampled_age_df['label'].astype(str)
random = sampled_age_df.sample(frac=1)
random

,text,label
196,goddess,__label__0
76029,never end cycl too,__label__1
82705,mayb thi can help,__label__1
64925,you ever see thi love you and wish you the best,__label__0
71010,thi same teacher who also chang her grade poli...,__label__0
...,...,...
40793,there profession turn the wind encourag you mo...,__label__1
92414,betray own dog,__label__0
24757,lmfao,__label__1
40530,noth cure hangov like svu marathon,__label__1


In [12]:
random.iloc[0:int(len(random)*0.8)].to_csv('/content/drive/MyDrive/CS640/final_project/age_train.csv', sep='\t', index = False, header = False, columns=['label', 'text'])
random.iloc[int(len(random)*0.8):int(len(random)*0.9)].to_csv('/content/drive/MyDrive/CS640/final_project/age_test.csv', sep='\t', index = False, header = False, columns=['label', 'text'])
random.iloc[int(len(random)*0.9):].to_csv('/content/drive/MyDrive/CS640/final_project/age_dev.csv', sep='\t', index = False, header = False, columns=['label', 'text']);

In [13]:
# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

corpus: Corpus = ClassificationCorpus(Path('/content/drive/MyDrive/CS640/final_project'), 
                            test_file='age_test.csv', 
                            dev_file='age_dev.csv', 
                            train_file='age_train.csv')
label_dict = corpus.make_label_dictionary()

# init BERT base (cases)
#optional_embedding = BertEmbeddings('bert-base-cased')
# OR init ELMo (original)
optional_embedding = ELMoEmbeddings('original')
# 3. make a list of word embeddings
#word_embeddings = [WordEmbeddings('glove')]
word_embeddings = list(filter(None, [
    optional_embedding,
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]))

# 3. initialize transformer document embeddings (many models are available)
#document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)
document_embeddings = DocumentRNNEmbeddings(
    word_embeddings,
    hidden_size=512,
    reproject_words=True,
    reproject_words_dimension=256,
)

# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('/content/drive/MyDrive/CS640/final_project',
              learning_rate=0.0003, # use very small learning rate
              mini_batch_size=15,
              max_epochs=8, # terminate after 5 epochs
              )

2021-12-06 04:35:09,467 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-forward-v0.1.pt not found in cache, downloading to /tmp/tmpy9740ft5


100%|██████████| 172513724/172513724 [00:10<00:00, 17144958.73B/s]

2021-12-06 04:35:20,082 copying /tmp/tmpy9740ft5 to cache at /root/.flair/embeddings/lm-jw300-forward-v0.1.pt


2021-12-06 04:35:20,300 removing temp file /tmp/tmpy9740ft5
2021-12-06 04:35:34,178 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-backward-v0.1.pt not found in cache, downloading to /tmp/tmpz9gw9c2w


100%|██████████| 172513724/172513724 [00:10<00:00, 17009832.84B/s]

...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


2021-12-06 06:16:45,781 epoch 8 - iter 385/3853 - loss 0.67228910 - samples/sec: 76.79 - lr: 0.000150
2021-12-06 06:17:59,821 epoch 8 - iter 770/3853 - loss 0.67072432 - samples/sec: 79.08 - lr: 0.000150
2021-12-06 06:19:14,470 epoch 8 - iter 1155/3853 - loss 0.66974651 - samples/sec: 77.96 - lr: 0.000150
2021-12-06 06:20:29,314 epoch 8 - iter 1540/3853 - loss 0.67001863 - samples/sec: 78.68 - lr: 0.000150
2021-12-06 06:21:44,988 epoch 8 - iter 1925/3853 - loss 0.66949555 - samples/sec: 76.80 - lr: 0.000150
2021-12-06 06:23:01,403 epoch 8 - iter 2310/3853 - loss 0.66939780 - samples/sec: 76.15 - lr: 0.000150
2021-12-06 06:24:18,561 epoch 8 - iter 2695/3853 - loss 0.66939932 - samples/sec: 75.79 - lr: 0.000150
2021-12-06 06:25:31,770 epoch 8 - iter 3080/3853 - loss 0.66900217 - samples/sec: 79.93 - lr: 0.000150
2021-12-06 06:26:47,009 epoch 8 - iter 3465/3853 - loss 0.66890237 - samples/sec: 77.66 - lr: 0.000150
2021-12-06 06:28:03,291 epoch 8 - iter 3850/3853 - loss 0.66879597 - sample

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


2021-12-06 06:31:15,120 	0.5528
2021-12-06 06:31:15,123 
Results:
- F-score (micro) 0.5528
- F-score (macro) 0.5523
- Accuracy 0.5528

By class:
              precision    recall  f1-score   support

           0     0.5608    0.5751    0.5678      3690
           1     0.5441    0.5296    0.5367      3533

   micro avg     0.5528    0.5528    0.5528      7223
   macro avg     0.5524    0.5523    0.5523      7223
weighted avg     0.5526    0.5528    0.5526      7223
 samples avg     0.5528    0.5528    0.5528      7223

2021-12-06 06:31:15,126 ----------------------------------------------------------------------------------------------------


{'dev_loss_history': [0.6892937421798706,
  0.6799957752227783,
  0.6814371943473816,
  0.6823769211769104,
  0.6780411005020142,
  0.6782018542289734,
  0.6775134205818176,
  0.6791810393333435],
 'dev_score_history': [0.5262,
  0.5514,
  0.5431,
  0.5427,
  0.5487,
  0.5466,
  0.5517,
  0.552],
 'test_score': 0.5528,
 'train_loss_history': [0.7147471720974939,
  0.6848269874573744,
  0.68072824836427,
  0.6788346228462178,
  0.6766975680141365,
  0.6758834069312902,
  0.6706568488912585,
  0.6687904705544617]}